The goal of this notebook is to pull in spreadsheets from reverie books and provide information on turns and provide insight on what is selling and what isnt.

Cells can be run individually by clicking into them and hitting shift + enter.



In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import datetime
!pip install xlsxwriter
import xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 1.8 MB/s eta 0:00:00


In [ ]:
import os
os.getcwd()
os.getcwdb()
dir = os.listdir()
archive = False #True/False toggles archive feature. Can archive input files
for item in dir:
  if archive:
    if item not in archive.listdir():
      #copy item to /archive
      #do this before renaming the files
      #only copy if item isn't already there
      print('item not in archive')

  if 'catalog' in item:
    #toggle-able archiving
    #copy item with original name to archive
    catalog = pd.read_excel(item)
    print('Catalog: ' + item)
  elif 'items' in item:
    weekly = pd.read_csv(item)
    print('\nWeekly: '+ item)
  elif 'Inventory' in item:
    sellthru = pd.read_excel(item)
    print('\nsellthru: ' + item)


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Catalog: catalog-2024-04-13 2_58 pm.xlsx

Weekly: items-2024-04-09-2024-04-13.csv


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")



sellthru: Inventory-sell-through-report-04-12-2024.xlsx


fix catalog's first row being blank / column names not being where they belong

In [ ]:
#data cleaning - update catagory columns to have the same capitalization
#possibly we can write this out bu changing .category to .Category elsewhere
catalog.rename(columns = {'Category':'category'}, inplace = True)
weekly.rename(columns = {'Category':'category'}, inplace = True)
#data cleaning - dropping rows with duplicate GTIN
catalog.drop_duplicates(subset='GTIN', inplace=True)
weekly.drop_duplicates(subset='GTIN', inplace=True)

In [ ]:
#data cleaning - category dictionary for clean_categories
#NewCategory : ['These Categories ', 'Are Combined ', 'In NewCategory]
#this is used to clean categories for Reverie Books
cat_dict = {
    'Children': ['Children', 'Children - Board', 'Children - GN', 'Children - LGBTQ', 'Children - Nonfiction', 'Childrens'],
    'Cooking': ['Cooking',  'Cooking - Spirits'],
    'Creative': ['Creative - Art', 'Creative - Writing'],
    'Fantasy': ['Fantasy', 'Fantasy - LGBTQ'],
    'Fiction': ['Fiction', 'Fiction - Asian', 'Fiction - Black', 'Fiction - Hispanic', 'Fiction - Historical', 'Fiction - Indigenous', 'Fiction - LGBTQ', 'Fiction - Literary', 'Fiction - ShortStories', 'Fiction - Western', 'Ficton', 'Indigenous - Fiction', 'LGBTQ - Fiction'],
    'GN': ['GN', 'GN - LGBTQ', 'GN - YA'],
    'History': ['History', 'History - Black', 'History - Immigration', 'History - LGBTQ', 'History - Native', 'History - Women', 'LGBTQ - History'],
    'Holiday': ['Holiday - Children - Easter', 'Holiday - Children - Halloween', 'Holiday - Children - Jewish', 'Holiday - Christmas', 'Holiday - Christmas - Children', 'Holiday - Christmas - LGBTQ', 'Holiday - Christmas - Romance', 'Holiday - Halloween - Children', 'Holiday - Hannukah', 'Holiday - Hanukkah - Children', 'Holiday - Kwanzaa - Children', 'Holiday - Spanish', 'Holiday - Thanksgiving'],
    'LGBT': ['LGBTQ', 'LGBTQ - Horror', 'LGBTQ - Memoir', 'LGBTQ - Nonfiction', 'LGBTQ - Trans'],
    'Memoir': ['Memoir', 'Memoir - Essays', 'Memoir - LGBTQ', 'Memoir - Music', 'Memoir - Pets', 'Memoir - Travel'],
    'MiddleReader': ['MiddleReader', 'MiddleReader - Fantasy', 'MiddleReader - GN', 'MiddleReader - LGBTQ', 'MiddleReader - Nonfiction', 'MiddlerReader - GN'],
    'Nonfiction': ['Nonfiction', 'Nonfiction - Abortion', 'Nonfiction - Black', 'Nonfiction - Crime', 'Nonfiction - Cultural', 'Nonfiction - Feminist', 'Nonfiction - Hispanic', 'Nonfiction - Immigration', 'Nonfiction - Jewish', 'Nonfiction - LGBTQ', 'Nonfiction - Native', 'Nonfiction - Police', 'Nonfiction - Politics', 'Nonfiction - Pop Culture', 'Nonfiction - SJ', 'Nonfiction - War', 'Romantasy'],
    'Parenting': ['Parenting', 'Parenting - LGBTQ', 'Parenting - Pregnancy', 'Parenting - SexEd'],
    'Poetry': ['Poetry', 'Poetry - LGBTQ', 'Poetry - Literary Journal'],
    'Regional': ['Regional', 'Regional - Fiction', 'Regional - Nonfiction'],
    'Romance': ['Romance', 'Romance - LGBTQ', 'Romance - LGBTQ - Gay', 'Romance - LGBTQ - Lesbian', 'Romance - LGBTQ - Queer', 'Romance - LGTBQ - Lesbian', 'Romance - Straight'],
    'Science': ['Science', 'Science - Anthropology', 'Science - climate', 'Science - Conservation', 'Science - Creativity', 'Science - Environment', 'Science - Evolution', 'Science - Nature', 'Science - Racism'],
    'SciFi': ['SciFi', 'SciFi - AfroFuturism', 'SciFi - Anthology', 'SciFi - BiPoc', 'SciFi - Classics', 'SciFi - Fantasy', 'SciFi - LGBTQ', 'SciFi - PostApocalyptic'],
    'SelfHelp': ['SelfHelp', 'SelfHelp - Addiction', 'SelfHelp - Astrology', 'SelfHelp - Black', 'SelfHelp - Business', 'SelfHelp - Communication', 'SelfHelp - Divorce', 'SelfHelp - Grief', 'SelfHelp - Health', 'SelfHelp - LGBTQ', 'SelfHelp - Love', 'SelfHelp - Metaphysics', 'SelfHelp - SJ', 'SelfHelp - Spirituality', 'SelfHelp - Trauma'],
    'SJ': ['SJ', 'SJ - Abortion', 'SJ - Black', 'SJ - Disability', 'SJ - Education', 'SJ - Feminism', 'SJ - Hispanic', 'SJ - History', 'SJ - Music', 'SJ - Native', 'SJ - Native - Nonfiction', 'SJ - Nonfiction', 'Social Justice', 'SocialJustice', 'SocialJustice - Feminism'],
    'Spanish': ['Spanish', 'Spanish - Children', 'Spanish - Fiction', 'Spanish - MiddleReader', 'Spanish - SciFi', 'Spanish - YA'],
    'Thriller': ['Thriller', 'Thriller - Horror', 'Thriller - Mystery', 'Thriller - Native', 'Mystery'],
    'Travel': ['Travel', 'Travel - Essays', 'Travel - LGTBQ'],
    'YA': ['YA', 'YA - Fantasy', 'YA - GN', 'YA - LGBTQ', 'YA - Nonfiction'],
    'Sideline': ['Sideline', 'Sideline - ActivityBook', 'Sideline - ArtSupplies', 'Sideline - Bookmarks', 'Sideline - Calendar', 'Sideline - Calendars', 'Sideline - Cards', 'Sideline - Creative', 'Sideline - DnD', 'Sideline - Edible', 'Sideline - Flag', 'Sideline - Games', 'Sideline - GiftBags', 'Sideline - Journals', 'Sideline - Lights', 'Sideline - Origami', 'Sideline - PensPencils', 'Sideline - Planner', 'Sideline - Puzzle', 'Sideline - Sketchbook', 'Sideline - Sketchbooks', 'Sideline - Stickers', 'Sideline - Tarot', 'Sideline - Tote', 'Sideline - Toys', 'Sideline - WrappingPaper', 'Sidelines', 'Sidelines - Tarot']
}

In [ ]:
#data cleaning - takes a dataframe and optional categories dictionary
#default categories dictionary is cat_dict
def clean_categories(df, categories=cat_dict):
  for x in df.index:
    for key, value in categories.items():
      if df.loc[x, 'category'] in value:
        df.loc[x, 'category'] = key
      #else:
        #df.loc[x, 'category'] = 'Other'


In [ ]:
#data cleaning - making smaller dataframes with only the columns we need
clean_categories(catalog)
clean_categories(weekly)
catalog_pruned = catalog[['Item Name', 'category', 'Price', 'Default Unit Cost', 'Current Quantity Reverie Books', 'Default Vendor Name', 'Variation Name', 'SKU', 'GTIN']]
weekly_pruned = weekly[['Item', 'Count', 'Price Point Name', 'SKU', 'GTIN']]
sellthru_pruned = sellthru[['GTIN', 'Last received', 'Last sold', 'On hand', 'On order', 'SKU']]
pruned_c_w = weekly_pruned.join(catalog_pruned.set_index('GTIN'), on='GTIN', lsuffix='_catalog', rsuffix='_weekly')
master_prune = pruned_c_w.join(sellthru_pruned.set_index('GTIN'), on='GTIN')
master_prune.dropna(axis=0, subset='GTIN', inplace=True)
#creates the master_sold df which shows the items that sold in weekly
master_sold = master_prune[master_prune['Count'] > 0]
master_sold = master_sold.assign(cogs=master_sold['Count'] * master_sold['Default Unit Cost'])
output_df = master_sold[['SKU', 'Item Name', 'Price', 'Default Unit Cost', 'category', 'Default Vendor Name', 'Variation Name', 'Last sold', 'Last received', 'On hand', 'On order']]
delta = output_df['Last sold'] - output_df['Last received']
output_df = output_df.assign(days_on_shelf=(delta).dt.days)

In [ ]:
# output_df = master_sold[['SKU', 'Item Name', 'Price', 'Default Unit Cost', 'category', 'Default Vendor Name', 'Variation Name', 'Last sold', 'Last received', 'On hand']]
# delta = output_df['Last sold'] - output_df['Last received']
# output_df = output_df.assign(days_on_shelf=(delta).dt.days)
writer = pd.ExcelWriter('output_formatted.xlsx', engine='xlsxwriter')
days_on_shelf = f'M2:M{output_df.shape[0]+1}' #to condition the whole days on shelf column
on_order = f'L2:L{output_df.shape[0]+1}'
output_df.to_excel(writer, sheet_name='Sheet1')
wb  = writer.book
worksheet = writer.sheets['Sheet1']
red = wb.add_format({'bg_color': 'red'})
green = wb.add_format({'bg_color': '77f963'})
worksheet.conditional_format(days_on_shelf, {'type':     'cell',
                                        'criteria': '>=',
                                        'value':    180,
                                        'format':   red})
worksheet.conditional_format(days_on_shelf, {'type':     'cell',
                                        'criteria': '<=',
                                        'value':    30,
                                        'format':   green})
worksheet.conditional_format(on_order, {'type':     'cell',
                                        'criteria': '>',
                                        'value':    0,
                                        'format':   red})
wb.close()

In [ ]:
#Run Before here

Cells below this point are development and aren't critical to functionality